## OPEN DATA - ECOBICI
En este notebook, realizaré un proceso de extracción de datos consultando a la API Transporte del Gobierno de la Ciudad Autónoma de Buenos Aires  sobre el estado actual de las estaciones de Ecobici (mas información en los siguientes links: [API Transporte](https://www.buenosaires.gob.ar/desarrollourbano/transporte/apitransporte), [API DOC](https://www.buenosaires.gob.ar/desarrollourbano/transporte/apitransporte/api-doc))
Luego, dichos datos continuaré con la manipulación y transformación de dichos datos utilizando Pandas


### Carga de datos
Dicha API devuelve los datos en formato JSON

In [1]:
import requests
from pprint import pprint
import pandas as pd


with open("credenciales.txt", "r") as file:
    client_id = file.readline().rstrip()
    client_secret = file.readline().rstrip()


api_url = "https://apitransporte.buenosaires.gob.ar/" 
resource_status = "ecobici/gbfs/stationStatus"
url = api_url + resource_status
url += "?client_id=" + client_id
url += "&client_secret=" + client_secret

r = requests.get(url)
stations_status = r.json()
r.status_code

200

In [2]:
pprint(stations_status)

{'data': {'stations': [{'is_charging_station': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1579966256,
                        'num_bikes_available': 1,
                        'num_bikes_available_types': {'ebike': 0,
                                                      'mechanical': 1},
                        'num_bikes_disabled': 0,
                        'num_docks_available': 19,
                        'num_docks_disabled': 0,
                        'station_id': '2',
                        'status': 'IN_SERVICE'},
                       {'is_charging_station': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1579966460,
                        'num_bikes_available': 0,
                        'num_bikes_available_types': 

Los datos sobre las estaciones son accedidos con la claves "data" y luego con "stations" para obtener una lista de JSON objects donde cada elemento es una estación determinada.

In [3]:
pprint(stations_status["data"]["stations"][0])

{'is_charging_station': False,
 'is_installed': 1,
 'is_renting': 1,
 'is_returning': 1,
 'last_reported': 1579966256,
 'num_bikes_available': 1,
 'num_bikes_available_types': {'ebike': 0, 'mechanical': 1},
 'num_bikes_disabled': 0,
 'num_docks_available': 19,
 'num_docks_disabled': 0,
 'station_id': '2',
 'status': 'IN_SERVICE'}


In [4]:
# Carga de dichos datos en un Dataframe
df_stations = pd.DataFrame(stations_status["data"]["stations"])
df_stations.head()

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status
0,2,1,"{'mechanical': 1, 'ebike': 0}",0,19,0,1,1,1,1579966256,False,IN_SERVICE
1,3,0,"{'mechanical': 0, 'ebike': 0}",0,20,0,1,1,1,1579966460,False,IN_SERVICE
2,4,0,"{'mechanical': 0, 'ebike': 0}",0,20,1,1,1,1,1579966384,False,IN_SERVICE
3,5,0,"{'mechanical': 0, 'ebike': 0}",1,39,0,1,1,1,1579966254,False,IN_SERVICE
4,6,3,"{'mechanical': 3, 'ebike': 0}",0,17,0,1,1,1,1579966214,False,IN_SERVICE


### Manipulación - Transformación

In [5]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 12 columns):
station_id                   396 non-null object
num_bikes_available          396 non-null int64
num_bikes_available_types    396 non-null object
num_bikes_disabled           396 non-null int64
num_docks_available          396 non-null int64
num_docks_disabled           396 non-null int64
is_installed                 396 non-null int64
is_renting                   396 non-null int64
is_returning                 396 non-null int64
last_reported                396 non-null int64
is_charging_station          396 non-null bool
status                       396 non-null object
dtypes: bool(1), int64(8), object(3)
memory usage: 34.5+ KB


Algunas columnas requieren un cambio en su tipo de datos

In [6]:
def change_type_column(df, **kwargs):
    """
    Función para convertir el tipo de datos de una columna
    donde cada nombre de los parámetros a pasar será una columna
    del DataFrame y su respectivo valor será el nuevo tipo de dato
    para esa columna.
    Retorna el mismo DataFrame con las columnas con sus nuevos tipos
    de datos
    """
    for col, a_type in kwargs.items():
        df[col] = df[col].astype(a_type)
    return df


df_stations = change_type_column(df_stations, 
                                 station_id='int64', 
                                 is_returning='bool', 
                                 is_renting='bool', 
                                 is_installed='bool')
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 12 columns):
station_id                   396 non-null int64
num_bikes_available          396 non-null int64
num_bikes_available_types    396 non-null object
num_bikes_disabled           396 non-null int64
num_docks_available          396 non-null int64
num_docks_disabled           396 non-null int64
is_installed                 396 non-null bool
is_renting                   396 non-null bool
is_returning                 396 non-null bool
last_reported                396 non-null int64
is_charging_station          396 non-null bool
status                       396 non-null object
dtypes: bool(4), int64(6), object(2)
memory usage: 26.4+ KB


Es necesario realizar una separación de los datos que contiene la columna "num_bikes_available_types" para una mejor representación de los mismos.

In [7]:
df_stations["num_mechanical_bikes_available"] = df_stations["num_bikes_available_types"].apply(lambda s : s["mechanical"])
df_stations["num_ebikes_available"] = df_stations["num_bikes_available_types"].apply(lambda s : s["ebike"])
df_stations = df_stations.drop("num_bikes_available_types", axis=1)
df_stations[["num_mechanical_bikes_available", "num_ebikes_available"]].head()

,num_mechanical_bikes_available,num_ebikes_available
0,1,0
1,0,0
2,0,0
3,0,0
4,3,0


Se crea un nuevo dataframe sobre las estaciones y su información como ubicación, nombre, etc.
El archivo en cuestión, ecobici_info.csv, se obtuvo en base al notebook ecobici_info.

In [8]:
df_stations_info = pd.read_csv('ecobici_info.csv')
df_stations_info.head()

,station_id,name,address,capacity
0,2,002 - Retiro I,"Ramos Mejia, Jose Maria, Dr. Av. & Del Liberta...",20
1,3,003 - ADUANA,Moreno & Av Paseo Colon,20
2,4,004 - Plaza Roma,Lavalle & Bouchard,20
3,5,005 - Plaza Italia,Av. Sarmiento 2601,40
4,6,006 - Parque Lezama,"Avenida Martin Garcia, 295",20


In [9]:
df_stations_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 4 columns):
station_id    379 non-null int64
name          379 non-null object
address       379 non-null object
capacity      379 non-null int64
dtypes: int64(2), object(2)
memory usage: 12.0+ KB


In [10]:
# Agregamos nuevas columnas al 1er dataframe realizando un merge con el df con info de las estaciones
pd.merge(df_stations_info, df_stations, on='station_id', how='inner')

,station_id,name,address,capacity,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,num_mechanical_bikes_available,num_ebikes_available
0,2,002 - Retiro I,"Ramos Mejia, Jose Maria, Dr. Av. & Del Liberta...",20,1,0,19,0,True,True,True,1579966256,False,IN_SERVICE,1,0
1,3,003 - ADUANA,Moreno & Av Paseo Colon,20,0,0,20,0,True,True,True,1579966460,False,IN_SERVICE,0,0
2,4,004 - Plaza Roma,Lavalle & Bouchard,20,0,0,20,1,True,True,True,1579966384,False,IN_SERVICE,0,0
3,5,005 - Plaza Italia,Av. Sarmiento 2601,40,0,1,39,0,True,True,True,1579966254,False,IN_SERVICE,0,0
4,6,006 - Parque Lezama,"Avenida Martin Garcia, 295",20,3,0,17,0,True,True,True,1579966214,False,IN_SERVICE,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,444,061-Ministerio de Economia,"Balcarce & Yrigoyen, Hipolito Av.",24,0,1,23,1,True,True,True,1579966321,False,IN_SERVICE,0,0
375,448,393 - Barrio 31,Carlos H. Perette 11,23,1,1,21,2,True,True,True,1579966485,False,IN_SERVICE,1,0
376,449,352 - San Jose de Flores,Avenida Rivadavia y Fray Cayetano,24,0,0,24,0,True,True,True,1579966480,False,IN_SERVICE,0,0
377,450,teste ecobici - 450,teste ecobici - 450,1,198,0,99,0,True,True,True,1567199965,False,IN_SERVICE,99,99


Cada vez que se ejecute por completo dicho notebook, se agregarán nuevas filas al archivo ecobici.csv

In [11]:
df_stations.to_csv('ecobici.csv', index=False, mode='a+')

De esta forma, se obtuvo un simple dataset sobre el estado de las estaciones de Ecobici.
Queda pendiente automatizar todo este proceso para ejecutarlo por ejemplo durante toda una semana en un rango horario específico a intervalos de una hora y así ir actualizando dicho dataset.